In [3]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

# --- 환경 변수 로드 ---
load_dotenv()
API_KEY = os.getenv('SEOUL_API_KEY')

# --- 1단계: API 데이터 수집 함수 ---
def collect_daily_data(date_str):
    """지정한 날짜의 모든 데이터를 API로 수집하여 DataFrame으로 반환"""
    all_data = []
    start_idx = 1
    
    print(f"[{date_str}] 데이터 수집 시작...")
    
    while True:
        # 1000개씩 나눠서 API 호출
        url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/tbCycleRentData/{start_idx}/{start_idx+999}/{date_str}/1"
        
        try:
            response = requests.get(url, timeout=30).json()
            
            # 'tbCycleRentData' 키가 없으면 데이터가 없는 것
            if 'tbCycleRentData' not in response:
                print("  - API 응답에 'tbCycleRentData'가 없어 중단합니다.")
                break
            
            data = response['tbCycleRentData']['row']
            all_data.extend(data)
            
            # 받아온 데이터가 1000개 미만이면, 그게 마지막 페이지
            if len(data) < 1000:
                print(f"  - 수집 완료 (총 {len(all_data):,} 건)")
                break
                
            start_idx += 1000
            
        except Exception as e:
            print(f"  - ⚠️ {start_idx} 인덱스에서 오류 발생: {e}")
            break
            
    # 비어있지 않다면 DataFrame으로 변환
    return pd.DataFrame(all_data) if all_data else None

# --- 1단계 실행 ---
# 딱 하루 날짜를 지정해서 수집
target_date = '2023-01-01'
raw_df = collect_daily_data(target_date)

if raw_df is not None and not raw_df.empty:
    # Parquet으로 로컬에 저장 (압축률이 좋음)
    output_filename = f"raw_data_{target_date}.parquet"
    raw_df.to_parquet(output_filename, compression='snappy', engine='pyarrow')
    
    print(f"\n✅ 로컬 저장 완료: {output_filename}")
    print(raw_df.head()) # 상위 5개 행 출력
else:
    print(f"\n⚠️ {target_date}에 수집된 데이터가 없습니다.")

[2023-01-01] 데이터 수집 시작...
  - API 응답에 'tbCycleRentData'가 없어 중단합니다.

⚠️ 2023-01-01에 수집된 데이터가 없습니다.


In [23]:
url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/tbCycleRentData/1/5/2022-10-01/0"
response = requests.get(url)

In [24]:
response.json()

{'rentData': {'list_total_count': '5550',
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'BIKE_ID': 'SPB-56517',
    'RENT_DT': '2022-10-01 00:00:26',
    'RENT_ID': '04802',
    'RENT_NM': '새우개마을',
    'RENT_HOLD': '0',
    'RTN_DT': '2022-10-01 00:00:35',
    'RTN_ID': '04802',
    'RTN_NM': '새우개마을',
    'RTN_HOLD': '0',
    'USE_MIN': '0',
    'USE_DST': '0.00',
    'USR_CLS_CD': 'USR_001',
    'SEX_CD': 'M',
    'BIRTH_YEAR': '2004',
    'RENT_STATION_ID': 'ST-2628',
    'RETURN_STATION_ID': 'ST-2628',
    'BIKE_SE_CD': '일반자전거',
    'START_INDEX': 0,
    'END_INDEX': 0,
    'RNUM': '1'},
   {'BIKE_ID': 'SPB-32505',
    'RENT_DT': '2022-10-01 00:00:39',
    'RENT_ID': '04557',
    'RENT_NM': '리버뷰신안인스빌2차 후문',
    'RENT_HOLD': '0',
    'RTN_DT': '2022-10-01 00:00:57',
    'RTN_ID': '04557',
    'RTN_NM': '리버뷰신안인스빌2차 후문',
    'RTN_HOLD': '0',
    'USE_MIN': '0',
    'USE_DST': '0.00',
    'USR_CLS_CD': 'USR_001',
    'BIRTH_YEAR': '1997',
    'RENT_STATION_ID': 

In [15]:
data = response.json()

In [ ]:
data

{'rentData': {'list_total_count': '2812',
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'BIKE_ID': 'SPB-51121',
    'RENT_DT': '2022-10-01 02:00:05',
    'RENT_ID': '01933',
    'RENT_NM': '개봉푸르지오아파트 상가',
    'RENT_HOLD': '0',
    'RTN_DT': '2022-10-01 02:01:13',
    'RTN_ID': '01933',
    'RTN_NM': '개봉푸르지오아파트 상가',
    'RTN_HOLD': '0',
    'USE_MIN': '1',
    'USE_DST': '0.00',
    'USR_CLS_CD': 'USR_001',
    'BIRTH_YEAR': '1999',
    'RENT_STATION_ID': 'ST-678',
    'RETURN_STATION_ID': 'ST-678',
    'BIKE_SE_CD': '일반자전거',
    'START_INDEX': 0,
    'END_INDEX': 0,
    'RNUM': '1'},
   {'BIKE_ID': 'SPB-81054',
    'RENT_DT': '2022-10-01 02:00:11',
    'RENT_ID': '04629',
    'RENT_NM': '한강초교보도육교 앞',
    'RENT_HOLD': '99',
    'RTN_DT': '2022-10-01 02:01:56',
    'RTN_ID': '04619',
    'RTN_NM': '한강트럼프월드3차 앞',
    'RTN_HOLD': '99',
    'USE_MIN': '1',
    'USE_DST': '370.00',
    'USR_CLS_CD': 'USR_001',
    'BIRTH_YEAR': '1988',
    'RENT_STATION_ID': 'ST-2456